# Demo of `cf-xarray`

In [1]:
import xarray as xr
import cf_xarray
import extract_model as em
import pandas as pd
from functools import partial


## Model output

We will use model output from several types of ocean models to demonstrate using the same commands with `cf-xarray`. The model output locations are based on what day it is, so here we defined "today".

In these examples, I read in the model output using a package called `extract_model` that knows the potential pitfalls of different models with the intention of mitigating those and adding attributes to the model output as needed. This is super important for `cf-xarray` to work correctly and is a work in progress. 

Another way to help `cf-xarray` work better is to use the built-in `ds.cf.guess_coord_axis(verbose=True)`.

In [2]:
today = pd.Timestamp.today()

### ROMS

In [3]:
try:
    url = ['https://opendap.co-ops.nos.noaa.gov/thredds/dodsC/TBOFS/fmrc/Aggregated_7_day_TBOFS_Fields_Forecast_best.ncd']
    ds_roms = xr.open_mfdataset(url, preprocess=em.preprocess)
except OSError:
    url = [today.strftime('https://opendap.co-ops.nos.noaa.gov/thredds/dodsC/NOAA/TBOFS/MODELS/%Y/%m/%d/nos.tbofs.fields.n001.%Y%m%d.t00z.nc'),
           today.strftime('https://opendap.co-ops.nos.noaa.gov/thredds/dodsC/NOAA/TBOFS/MODELS/%Y/%m/%d/nos.tbofs.fields.n002.%Y%m%d.t00z.nc')]
    ds_roms = xr.open_mfdataset(url, preprocess=em.preprocess)


oc_open: server error retrieving url: code=500 message="null"

### POM

In [5]:
try:
    url = ['https://opendap.co-ops.nos.noaa.gov/thredds/dodsC/LOOFS/fmrc/Aggregated_7_day_LOOFS_Fields_Forecast_best.ncd']
    ds_pom = xr.open_mfdataset(url, preprocess=em.preprocess)
except OSError:
    url = [today.strftime('https://opendap.co-ops.nos.noaa.gov/thredds/dodsC/NOAA/LOOFS/MODELS/%Y/%m/%d/glofs.loofs.fields.nowcast.%Y%m%d.t00z.nc'),
           today.strftime('https://opendap.co-ops.nos.noaa.gov/thredds/dodsC/NOAA/LOOFS/MODELS/%Y/%m/%d/glofs.loofs.fields.nowcast.%Y%m%d.t06z.nc')]
    ds_pom = xr.open_mfdataset(url, preprocess=em.preprocess)

ds_pom = ds_pom.assign_coords({'lat': ds_pom['lat'], 'lon': ds_pom['lon']})

oc_open: server error retrieving url: code=500 message="null"

### SELFE

In [7]:
url = [today.strftime('https://opendap.co-ops.nos.noaa.gov/thredds/dodsC/NOAA/CREOFS/MODELS/%Y/%m/%d/nos.creofs.fields.n000.%Y%m%d.t03z.nc'),
       today.strftime('https://opendap.co-ops.nos.noaa.gov/thredds/dodsC/NOAA/CREOFS/MODELS/%Y/%m/%d/nos.creofs.fields.n001.%Y%m%d.t03z.nc')]
ds_selfe = xr.open_mfdataset(url, preprocess=em.preprocess)
# 

### ...Use ROMS First

In [20]:
ds = ds_roms

`cf-xarray` description for whole Dataset

In [21]:
ds.cf

Coordinates:
- CF Axes: * X: ['nx']
           * Y: ['ny']
           * Z: ['sigma']
           * T: ['time']

- CF Coordinates:   longitude: ['lon']
                    latitude: ['lat']
                    vertical: ['z']
                  * time: ['time']

- Cell Measures:   area, volume: n/a

- Standard Names:   latitude: ['lat']
                    longitude: ['lon']
                  * ocean_sigma_coordinate: ['sigma']
                  * time: ['time']

- Bounds:   n/a

Data Variables:
- Cell Measures:   area, volume: n/a

- Standard Names:   depth: ['depth']
                    eastward_sea_water_velocity: ['u']
                    eastward_wind: ['air_u']
                    northward_sea_water_velocity: ['v']
                    northward_wind: ['air_v']
                    sea_surface_elevation: ['zeta']
                    sea_water_temperature: ['temp']

- Bounds:   n/a

## Basic Operations

### Selecting and Slicing

#### Time

Access time:

In [22]:
ds.cf['T']

<xarray.DataArray 'time' (time: 12)>
array(['2022-08-10T18:59:45.937500032', '2022-08-10T20:00:14.062499968',
       '2022-08-10T21:00:00.000000000', '2022-08-10T21:59:45.937500032',
       '2022-08-10T23:00:14.062499968', '2022-08-11T00:00:00.000000000',
       '2022-08-11T00:59:45.937500032', '2022-08-11T02:00:14.062499968',
       '2022-08-11T03:00:00.000000000', '2022-08-11T03:59:45.937500032',
       '2022-08-11T05:00:14.062499968', '2022-08-11T06:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2022-08-10T18:59:45.937500032 ... 2022-08-...
Attributes:
    long_name:      Time
    base_date:      [2006    1    1    0]
    standard_name:  time

Select time:

In [23]:
ds.cf.sel(T='2022-8-9T20', method='nearest').cf['T']

<xarray.DataArray 'time' ()>
array('2022-08-10T18:59:45.937500032', dtype='datetime64[ns]')
Attributes:
    long_name:      Time
    base_date:      [2006    1    1    0]
    standard_name:  time

Select time by index:

In [24]:
ds.cf.isel(T=0).cf['T']

<xarray.DataArray 'time' ()>
array('2022-08-10T18:59:45.937500032', dtype='datetime64[ns]')
Attributes:
    long_name:      Time
    base_date:      [2006    1    1    0]
    standard_name:  time

#### Horizontal Space Dimensions

You can use these in a meaningful way if your horizontal dimensions are your horizontal coordinates. That is, for example, that you have a vector of longitudes and latitudes that define your (rectilinear) grid instead of a 2D or unstructured grid. You can still do it in these later cases but it may not be as meaningful.

Slice 

In [25]:
ds.cf.isel(X=slice(0,10))

<xarray.Dataset>
Dimensions:    (time: 12, ny: 25, nx: 10, sigma: 20)
Coordinates:
  * time       (time) datetime64[ns] 2022-08-10T18:59:45.937500032 ... 2022-0...
    lon        (ny, nx) float32 dask.array<chunksize=(25, 10), meta=np.ndarray>
    lat        (ny, nx) float32 dask.array<chunksize=(25, 10), meta=np.ndarray>
  * sigma      (sigma) float32 0.0 0.0227 0.0454 0.0681 ... 0.8853 0.9534 1.0
  * nx         (nx) int64 0 1 2 3 4 5 6 7 8 9
  * ny         (ny) int64 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23 24
    z          (time, sigma, ny, nx) float32 dask.array<chunksize=(6, 20, 25, 10), meta=np.ndarray>
Data variables:
    validtime  (time) |S64 dask.array<chunksize=(6,), meta=np.ndarray>
    mask       (time, ny, nx) float32 dask.array<chunksize=(6, 25, 10), meta=np.ndarray>
    depth      (time, ny, nx) float32 dask.array<chunksize=(6, 25, 10), meta=np.ndarray>
    zeta       (time, ny, nx) float32 dask.array<chunksize=(6, 25, 10), meta=np.ndarray>
    air_u      (time, ny, nx) float32 dask.array<chunksize=(6, 25, 10), meta=np.ndarray>
    air_v      (time, ny, nx) float32 dask.array<chunksize=(6, 25, 10), meta=np.ndarray>
    u          (time, sigma, ny, nx) float32 dask.array<chunksize=(6, 20, 25, 10), meta=np.ndarray>
    v          (time, sigma, ny, nx) float32 dask.array<chunksize=(6, 20, 25, 10), meta=np.ndarray>
    temp       (time, sigma, ny, nx) float32 dask.array<chunksize=(6, 20, 25, 10), meta=np.ndarray>
Attributes: (12/17)
    datum1:                          reference to LWD=zeta
    datum2:                          reference to IGLD85=zeta - IGLD85
    file_type:                       Full_Grid
    Conventions:                     COARDS
    grid_type:                       rectilinear
    z_type:                          2-D
    ...                              ...
    history:                         Operation-CO-OPS
    references:                      greg.mott@noaa.gov
    creation_date:                   2022-08-11  0:50:18  00:00
    DODS.strlen:                     19
    DODS.dimName:                    validtime_len
    DODS_EXTRA.Unlimited_Dimension:  time

#### Variable

Select variable by `standard_name`:

In [26]:
var = 'sea_water_temperature'

ds.cf[var]

<xarray.DataArray 'temp' (time: 12, sigma: 20, ny: 25, nx: 61)>
dask.array<concatenate, shape=(12, 20, 25, 61), dtype=float32, chunksize=(6, 20, 25, 61), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2022-08-10T18:59:45.937500032 ... 2022-08-...
  * sigma    (sigma) float32 0.0 0.0227 0.0454 0.0681 ... 0.8853 0.9534 1.0
  * nx       (nx) int64 0 1 2 3 4 5 6 7 8 9 10 ... 51 52 53 54 55 56 57 58 59 60
  * ny       (ny) int64 0 1 2 3 4 5 6 7 8 9 10 ... 15 16 17 18 19 20 21 22 23 24
Attributes:
    long_name:      Temperature
    units:          Celsius
    standard_name:  sea_water_temperature

Create your own custom vocabulary for variables that can search the variable metadata with rules you write, which can use regular expressions.

In [27]:
criteria = {'temp': {'units': 'Celsius'}}

cf_xarray.set_options(custom_criteria=criteria)

ds.cf['temp']

<xarray.DataArray 'temp' (time: 12, sigma: 20, ny: 25, nx: 61)>
dask.array<concatenate, shape=(12, 20, 25, 61), dtype=float32, chunksize=(6, 20, 25, 61), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2022-08-10T18:59:45.937500032 ... 2022-08-...
  * sigma    (sigma) float32 0.0 0.0227 0.0454 0.0681 ... 0.8853 0.9534 1.0
  * nx       (nx) int64 0 1 2 3 4 5 6 7 8 9 10 ... 51 52 53 54 55 56 57 58 59 60
  * ny       (ny) int64 0 1 2 3 4 5 6 7 8 9 10 ... 15 16 17 18 19 20 21 22 23 24
Attributes:
    long_name:      Temperature
    units:          Celsius
    standard_name:  sea_water_temperature

### Plotting

Plot first time step at surface.

In [28]:
ds.cf[var].cf.isel(T=0).cf.isel(Z=0).cf.plot(x='longitude', y='latitude')

KeyError: 'longitude'

## AP Level

### Create Bounding Box

In [29]:
da = ds.cf[var].cf.isel(T=0, Z=-1)

dd = 1
lon0 = da.cf['longitude'].mean()
lat0 = da.cf['latitude'].mean()
bbox = [lon0 - dd, lat0 - dd, lon0 + dd, lat0 + dd]

box = (
    (bbox[0] < da.cf["longitude"])
    & (da.cf["longitude"] < bbox[2])
    & (bbox[1] < da.cf["latitude"])
    & (da.cf["latitude"] < bbox[3])
).compute()

da.where(box, drop=True).cf.plot(x='longitude', y='latitude')

KeyError: "DataArray.cf does not understand the key 'longitude'. Use 'repr(DataArray.cf)' (or 'DataArray.cf' in a Jupyter environment) to see a list of key names that can be interpreted."

### Decode Vertical Coords

In [30]:
outnames = { Z: Z.replace('s', 'z') for Z in ds.cf.axes['Z']}

ds.cf.decode_vertical_coords(outnames=outnames)
ds

<xarray.Dataset>
Dimensions:    (time: 12, ny: 25, nx: 61, sigma: 20)
Coordinates:
  * time       (time) datetime64[ns] 2022-08-10T18:59:45.937500032 ... 2022-0...
    lon        (ny, nx) float32 dask.array<chunksize=(25, 61), meta=np.ndarray>
    lat        (ny, nx) float32 dask.array<chunksize=(25, 61), meta=np.ndarray>
  * sigma      (sigma) float32 0.0 0.0227 0.0454 0.0681 ... 0.8853 0.9534 1.0
  * nx         (nx) int64 0 1 2 3 4 5 6 7 8 9 ... 51 52 53 54 55 56 57 58 59 60
  * ny         (ny) int64 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23 24
    z          (time, sigma, ny, nx) float32 dask.array<chunksize=(6, 20, 25, 61), meta=np.ndarray>
    zigma      (time, ny, nx, sigma) float32 dask.array<chunksize=(6, 25, 61, 20), meta=np.ndarray>
Data variables:
    validtime  (time) |S64 dask.array<chunksize=(6,), meta=np.ndarray>
    mask       (time, ny, nx) float32 dask.array<chunksize=(6, 25, 61), meta=np.ndarray>
    depth      (time, ny, nx) float32 dask.array<chunksize=(6, 25, 61), meta=np.ndarray>
    zeta       (time, ny, nx) float32 dask.array<chunksize=(6, 25, 61), meta=np.ndarray>
    air_u      (time, ny, nx) float32 dask.array<chunksize=(6, 25, 61), meta=np.ndarray>
    air_v      (time, ny, nx) float32 dask.array<chunksize=(6, 25, 61), meta=np.ndarray>
    u          (time, sigma, ny, nx) float32 dask.array<chunksize=(6, 20, 25, 61), meta=np.ndarray>
    v          (time, sigma, ny, nx) float32 dask.array<chunksize=(6, 20, 25, 61), meta=np.ndarray>
    temp       (time, sigma, ny, nx) float32 dask.array<chunksize=(6, 20, 25, 61), meta=np.ndarray>
Attributes: (12/17)
    datum1:                          reference to LWD=zeta
    datum2:                          reference to IGLD85=zeta - IGLD85
    file_type:                       Full_Grid
    Conventions:                     COARDS
    grid_type:                       rectilinear
    z_type:                          2-D
    ...                              ...
    history:                         Operation-CO-OPS
    references:                      greg.mott@noaa.gov
    creation_date:                   2022-08-11  0:50:18  00:00
    DODS.strlen:                     19
    DODS.dimName:                    validtime_len
    DODS_EXTRA.Unlimited_Dimension:  time

In [31]:
ds.cf

Coordinates:
- CF Axes: * X: ['nx']
           * Y: ['ny']
           * Z: ['sigma']
           * T: ['time']

- CF Coordinates:   longitude: ['lon']
                    latitude: ['lat']
                    vertical: ['z', 'zigma']
                  * time: ['time']

- Cell Measures:   area, volume: n/a

- Standard Names:   latitude: ['lat']
                    longitude: ['lon']
                  * ocean_sigma_coordinate: ['sigma']
                    sea_surface_elevation: ['zigma']
                  * time: ['time']

- Bounds:   n/a

Data Variables:
- Cell Measures:   area, volume: n/a

- Standard Names:   depth: ['depth']
                    eastward_sea_water_velocity: ['u']
                    eastward_wind: ['air_u']
                    northward_sea_water_velocity: ['v']
                    northward_wind: ['air_v']
                    sea_surface_elevation: ['zeta']
                    sea_water_temperature: ['temp']

- Bounds:   n/a